## On this notebook we extract Censo 2010 individual data from their files.


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
import os

if not os.path.exists('./../../samplerCensoARG/'):
    !git clone https://github.com/matuteiglesias/samplerCensoARG.git ./../../

In [12]:
!python ./../../samplerCensoARG/samplear.py -dbp '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub' -f 0.01 -y 2003 2005 -n ARG

..
['/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub']
[0.01]
['ARG']
[2003, 2005]
total_pais
[########################################] | 100% Completed | 45.1s
2003
[                                        ] | 0% Completed |  3.3s^C
[                                        ] | 0% Completed |  3.4s
Traceback (most recent call last):
  File "./../../samplerCensoARG/samplear.py", line 118, in <module>
    table = tabla_censo.compute()
  File "/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/base.py", line 167, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/base.py", line 452, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/threaded.py", line 76, in get
    results = get_async(
  File "/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/local.py", line 475, in get_async
    key, res_info, failed = queue_g

In [10]:
import pandas as pd
pd.read_csv('/home/miglesia/repositories/indice-pobreza-ExactasUBA/notebooks/../data/info/AGLO_rk/')

NotADirectoryError: [Errno 20] Not a directory: '/home/miglesia/repositories/indice-pobreza-ExactasUBA/notebooks/../data/info/AGLO_rk/'

In [4]:

for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    table = pd.read_csv('./../data/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv')
    
    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    ## VIVIENDA
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    ## HOGAR
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
    # PERSONA
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

    table.to_csv('./../data/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv', index = False)


NameError: name 'startyr' is not defined